# 🧠 SVM-Based VLSI Partitioning Model

This notebook implements a Support Vector Machine (SVM)-based approach for partitioning VLSI netlists represented as graphs.

### Model Logic:
- Each node in the graph represents a circuit component with features:
  - Power, Area, Average Edge Distance, Degree
- Pseudo-labels for supervised learning are generated using **KMeans** clustering.
- An **SVM classifier** is trained on the node features and labels.
- After training, each node is assigned to a partition, and standard partitioning metrics are computed.

### Goals:
- Minimize inter-partition **cut edges**
- Minimize total **wire length**
- Balance **power and area** across partitions
- Reduce **critical path delay**


In [1]:
# SVM Partitioning Model
# Uses pseudo-labels from KMeans and trains an SVM classifier on node features
# Features used: [power, area, avg_edge_distance, degree]

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
import random
import math


In [2]:
# Graph Gen 1.2
# Added inputs and outputs
# The power consumption for a node ranges from 1 to 50 units and uniformly distributed.
# Plus the area may vary from 1 to 5 units as well, uniformly distributed
# Ensured the following:
# The input nodes must have at least one outgoing edge and no incoming edge
# The output nodes must have one incoming edge only
# Internal nodes must have at least one incoming and at least one outgoing edge

def generate_netlist(
    num_nodes=50,
    num_edges=100,
    enable_area=True,
    enable_power=True,
    enable_wire_count=True,
    enable_distance=True,
    seed=42
):
    G = nx.DiGraph()
    num_inputs = math.ceil(math.log2(num_nodes))
    num_outputs = math.ceil(num_inputs / 2)
    input_nodes = [f"IN_{i}" for i in range(num_inputs)]
    output_nodes = [f"OUT_{i}" for i in range(num_outputs)]
    internal_nodes = [f"N_{i}" for i in range(num_nodes)]

    for node in internal_nodes:
        G.add_node(node)
        if enable_area:
            G.nodes[node]['area'] = round(random.uniform(1.0, 5.0), 2)
        if enable_power:
            G.nodes[node]['power'] = round(random.uniform(1.0, 50.0), 2)

    for node in input_nodes + output_nodes:
        G.add_node(node)

    for input_node in input_nodes:
        target = random.choice(internal_nodes)
        G.add_edge(input_node, target)

    for output_node in output_nodes:
        source = random.choice(internal_nodes)
        G.add_edge(source, output_node)

    for node in internal_nodes:
        if G.in_degree(node) == 0:
            source = random.choice(input_nodes + internal_nodes)
            G.add_edge(source, node)
        if G.out_degree(node) == 0:
            target = random.choice(internal_nodes + output_nodes)
            G.add_edge(node, target)

    existing_edges = set(G.edges())
    while len(G.edges()) < num_edges:
        u, v = random.sample(internal_nodes, 2)
        if u != v and (u, v) not in existing_edges:
            G.add_edge(u, v)
            existing_edges.add((u, v))

    for u, v in G.edges():
        if enable_wire_count:
            G.edges[u, v]['wires'] = random.randint(1, 5)
        if enable_distance:
            G.edges[u, v]['distance'] = round(random.uniform(1.0, 10.0), 2)

    return G, input_nodes, output_nodes


In [3]:
# Borrowed from KMeans_1d.ipynb
# Computes number of inter-cluster cuts, wire length, and delay

def compute_wire_metrics(G, cluster_labels, input_nodes, output_nodes):
    num_cuts = 0
    total_wire_length = 0
    G_weighted = nx.DiGraph()

    for u, v in G.edges():
        wire_length = G.edges[u, v].get("distance", 1)
        cluster_u, cluster_v = cluster_labels.get(u), cluster_labels.get(v)

        if cluster_u == cluster_v:
            edge_weight = wire_length
        else:
            edge_weight = wire_length * 10
            num_cuts += 1

        total_wire_length += edge_weight
        G_weighted.add_edge(u, v, weight=edge_weight)

    while not nx.is_directed_acyclic_graph(G_weighted):
        try:
            cycle = next(nx.simple_cycles(G_weighted))
            min_edge = min(
                ((cycle[i], cycle[(i + 1) % len(cycle)]) for i in range(len(cycle))),
                key=lambda e: G_weighted.edges[e].get("weight", 1)
            )
            G_weighted.remove_edge(*min_edge)
        except StopIteration:
            break

    critical_length = 0
    for input_node in input_nodes:
        if input_node in G_weighted.nodes:
            for output_node in output_nodes:
                if output_node in G_weighted.nodes:
                    try:
                        path = nx.dag_longest_path(G_weighted, weight="weight")
                        path_length = sum(G_weighted[u][v]["weight"] for u, v in zip(path, path[1:]))
                        critical_length = max(critical_length, path_length)
                    except nx.NetworkXNoPath:
                        pass

    return num_cuts, total_wire_length, critical_length


In [4]:
# SVM Partition Model
# Uses pseudo-labels from KMeans to train binary SVM classifier

def extract_node_features(G):
    features = []
    node_ids = []
    for node in G.nodes():
        if node.startswith("IN_") or node.startswith("OUT_"):
            continue

        power = G.nodes[node].get("power", 0)
        area = G.nodes[node].get("area", 0)
        in_edges = G.in_edges(node)
        out_edges = G.out_edges(node)
        edge_distances = [G.edges[u, v].get("distance", 1.0) for u, v in list(in_edges) + list(out_edges)]
        avg_distance = np.mean(edge_distances) if edge_distances else 0.0
        degree = G.in_degree(node) + G.out_degree(node)

        features.append([power, area, avg_distance, degree])
        node_ids.append(node)

    return np.array(features), node_ids

def svm_partition_model(G,k):
    features, node_ids = extract_node_features(G)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)

    kmeans = KMeans(n_clusters=k, n_init=10)
    y_pseudo = kmeans.fit_predict(X_scaled)

    clf = SVC(kernel='rbf', C=1.0, gamma='scale', decision_function_shape='ovo')  # ovo = one-vs-one (default)
    clf.fit(X_scaled, y_pseudo)
    y_pred = clf.predict(X_scaled)

    return {node_ids[i]: int(y_pred[i]) for i in range(len(node_ids))}


# This is the driver code that will implement multi class SVM and return the metrics
def svm_model_driver(graph, inputs, outputs, k):

  cluster_labels = svm_partition_model(graph,k)
  num_cuts, total_wire_length, critical_length = compute_wire_metrics(graph, cluster_labels, inputs, outputs)
  return num_cuts, critical_length


In [5]:
# Cluster Visualization

def visualize_clusters(graph, cluster_labels):
    pos = nx.spring_layout(graph, seed=42)
    clusters = set(cluster_labels.values())
    colors = plt.cm.tab20(np.linspace(0, 1, len(clusters)))

    for i, cluster in enumerate(clusters):
        nodes = [node for node in graph.nodes if cluster_labels.get(node, -1) == cluster]
        nx.draw_networkx_nodes(graph, pos, nodelist=nodes, node_color=[colors[i]], label=f'Cluster {cluster}')

    nx.draw_networkx_edges(graph, pos, alpha=0.4)
    nx.draw_networkx_labels(graph, pos, font_size=8)
    plt.title("SVM Partitioning of VLSI Netlist")
    plt.axis('off')
    plt.legend()
    plt.show()

In [6]:
# Example run
if __name__ == "__main__":
    min_nodes = 50
    max_nodes = 1000
    step = 10
    num_runs = 1
    
    print(f"Running scaling analysis from {min_nodes} to {max_nodes} nodes...")
    node_counts, avg_cuts, avg_wire_lengths, avg_critical_lengths = analyze_scaling(
        min_nodes, max_nodes, step, num_runs
    )
    
    print("Analysis complete, generating plots...")
    plot_scaling_results(node_counts, avg_cuts, avg_wire_lengths, avg_critical_lengths)
    
    # Print some numerical results
    print("\nNumerical Results:")
    for i, nodes in enumerate(node_counts):
        if avg_cuts[i] is not None:
            print(f"Nodes: {nodes:3d}, Cuts: {avg_cuts[i]:6.2f}, Wire Length: {avg_wire_lengths[i]:8.2f}, Critical Length: {avg_critical_lengths[i]:8.2f}")

Running scaling analysis from 50 to 1000 nodes...


NameError: name 'analyze_scaling' is not defined